In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import csv,json
from IPython.display import display

In [2]:
a = pd.read_csv('FLOWS.csv')
b = pd.read_csv('FLOWS2.csv')

In [3]:
display(a.head())

,Flow Type,Member,Description,Transaction Value Date,Amount,Original Disbursement Date,Original Arrangement Date
0,PRGT Disbursements,"Afghanistan, Islamic Republic of",Extended Credit Facility,1/19/07,13200000,1/19/07,6/26/06
1,PRGT Disbursements,"Afghanistan, Islamic Republic of",Extended Credit Facility,3/29/07,11300000,3/29/07,6/26/06
2,PRGT Interest,"Afghanistan, Islamic Republic of",PRGT Interest,6/29/07,43353,NaN,NaN
3,PRGT Disbursements,"Afghanistan, Islamic Republic of",Extended Credit Facility,7/23/07,11300000,7/23/07,6/26/06
4,PRGT Interest,"Afghanistan, Islamic Republic of",PRGT Interest,12/28/07,85541,NaN,NaN


In [4]:
def compare(a,b):
    return (a == b) | ((a != a) & (b != b))
res = compare(a,b)
res.sum()/len(a)

Flow Type                     1.000000
Member                        1.000000
Description                   1.000000
Transaction Value Date        0.999961
Amount                        0.999726
Original Disbursement Date    1.000000
Original Arrangement Date     0.999883
dtype: float64

In [5]:
res.all()

Flow Type                      True
Member                         True
Description                    True
Transaction Value Date        False
Amount                        False
Original Disbursement Date     True
Original Arrangement Date     False
dtype: bool

In [6]:
columns = a.select_dtypes(include='number').columns
res = np.abs(a[columns] - b[columns]).max()
res

Amount    9000000
dtype: int64

In [7]:
merge_on = ['Flow Type','Member','Transaction Value Date']

In [8]:
tmp0 = a.loc[:, merge_on]
tmp1 = b.loc[:, merge_on]

tmp0['count'] = tmp0.groupby(by=merge_on).cumcount()
tmp1['count'] = tmp1.groupby(by=merge_on).cumcount()
tmp0['index'] = a.index
tmp1['index'] = b.index
res = pd.merge(
    tmp0,
    tmp1,
    how='outer',
    on=merge_on + ['count'],
    indicator=True,
    suffixes=[
        '_file0',
        '_file1'],
    validate='one_to_one')

In [9]:
merge_idx0 = pd.Index(res[res['_merge'] == 'both']['index_file0'], dtype='int64').rename(None)
merge_idx1 = pd.Index(res[res['_merge'] == 'both']['index_file1'], dtype='int64').rename(None)

left_only = pd.Index(res[res['_merge'] == 'left_only']['index_file0'], dtype='int64').rename(None)
right_only = pd.Index(res[res['_merge'] == 'right_only']['index_file1'], dtype='int64').rename(None)

ta = a.loc[merge_idx0]
tb = b.loc[merge_idx1]
resdf = compare(ta.reset_index(drop=True),tb.reset_index(drop=True))
diffres = ~resdf[~resdf.all(axis=1)]

diffres['idx0'] = merge_idx0[diffres.index]
diffres['idx1'] = merge_idx1[diffres.index]
diffres.set_index(['idx0', 'idx1'], inplace=True)

In [10]:
print('Missing Data')
display(a.loc[left_only][:10])

Missing Data


,Flow Type,Member,Description,Transaction Value Date,Amount,Original Disbursement Date,Original Arrangement Date
1223,GRA Repurchases,Argentina,Extended Fund Facility,3/27/02,21348667,3/29/96,3/31/92


In [11]:
print('New Data')
display(b.loc[right_only][:10])

New Data


,Flow Type,Member,Description,Transaction Value Date,Amount,Original Disbursement Date,Original Arrangement Date
1222,GRA Repurchases,Argentina,Extended Fund Facility,3/25/02,21348583,9/29/95,3/31/92


In [12]:
def row_diff(diff_data, a, b):
    def display_diff(r):
        display_columns = ['Flow Type','Member','Transaction Value Date'] + r[r].index.tolist()
        res_row = pd.DataFrame(
            {f'file0': a.loc[r.name[0], display_columns], f'file1': b.loc[r.name[1], display_columns]})
        return res_row.transpose()

    return diff_data.apply(display_diff, axis=1)
for row in row_diff(diffres.head(2), a, b):
    display(row)

,Flow Type,Member,Transaction Value Date,Amount
file0,PRGT Disbursements,"Afghanistan, Islamic Republic of",4/29/09,11300000
file1,PRGT Disbursements,"Afghanistan, Islamic Republic of",4/29/09,11300001


,Flow Type,Member,Transaction Value Date,Amount
file0,PRGT Repayments,"Afghanistan, Islamic Republic of",8/28/13,1130000
file1,PRGT Repayments,"Afghanistan, Islamic Republic of",8/28/13,1130005


In [13]:
display(a.apply(pd.Series.value_counts).fillna(0))

,Flow Type,Member,Description,Transaction Value Date,Amount,Original Disbursement Date,Original Arrangement Date
-3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
-1,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,0.0,0.0,6.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
Venezuela,0.0,37.0,0.0,0.0,0.0,0.0,0.0
Vietnam,0.0,168.0,0.0,0.0,0.0,0.0,0.0
"Yemen, Republic of",0.0,293.0,0.0,0.0,0.0,0.0,0.0
Zambia,0.0,299.0,0.0,0.0,0.0,0.0,0.0


In [14]:
freq_a = a.apply(pd.Series.value_counts).fillna(0)
freq_b = b.apply(pd.Series.value_counts).fillna(0)
res = freq_a.sub(freq_b,fill_value=0)
res = res.abs()

In [15]:
res[res.any(axis=1)].head()

,Flow Type,Member,Description,Transaction Value Date,Amount,Original Disbursement Date,Original Arrangement Date
101458,0.0,0.0,0.0,0.0,1.0,0.0,0.0
102458,0.0,0.0,0.0,0.0,1.0,0.0,0.0
233400,0.0,0.0,0.0,0.0,1.0,0.0,0.0
235400,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1130000,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
res[res.any(axis=1)].tail()

,Flow Type,Member,Description,Transaction Value Date,Amount,Original Disbursement Date,Original Arrangement Date
5/22/91,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5/22/95,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5/22/96,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6/7/10,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6/7/11,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
res.sum()

Flow Type                      0.0
Member                         0.0
Description                    0.0
Transaction Value Date         2.0
Amount                        14.0
Original Disbursement Date     0.0
Original Arrangement Date      6.0
dtype: float64